In [1]:
import os
import math
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer
import time
import gurobipy as gp
    

def fill_variables(filename, file_path, file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, v_capacity, capacities_depots,
        customers_demands, opening_costs_depots, opening_cost_route, zero_or_not):
    
    line = next(file)
    n_customers = int(line.strip())
    print("Número de customers: " + str(n_customers))
    line = next(file)
    n_available_depots = int(line.strip())
    print("Número de depots disponibles: " + str(n_available_depots))
    line = next(file)
    line = next(file)
            
    count = n_available_depots
    #coordenadas depots
    while(line.strip() and count > 0):
        #print(line.split())
        coordinates_depots.append(line.split())    
        line = next(file)
        count = count -1
            
    #print("Coordenadas depots:")
    #print(coordinates_depots)
    if file_path != "Inst2":
        line = next(file)
    if filename == "coordDas150.dat":
        line = next(file)
    #Coordenadas customers
    while(line.strip()):
        coordinates_customers.append(line.split())
        line = next(file)
            
    #print("Coordenadas customers:")
    #print(coordinates_customers)
            
    line = next(file)
            
    #Capacidad de vehículos
    v_capacity = int(line.strip())
    #print("Capacidad vehículos: " + v_capacity)
    line = next(file)
    line = next(file)
            
    #Capacidad de los depots
    while(line.strip()):
        capacities_depots.append(line.split())
                
        line = next(file)
            
    #print("Capacidades depots:")
    #print(capacities_depots)
            
    line = next(file)
            
    while(line.strip()):
        customers_demands.append(line.strip())
                
        line = next(file)
            
    #print("Demandas customers:")
    #print(customers_demands)
            
    line = next(file)
            
    while(line.strip()):
        opening_costs_depots.append(line.split())
                
        line = next(file)
            
    #print("Costos depots:")
    #print(opening_costs_depots)
            
    line = next(file)
            
            
    opening_cost_route = float(line.strip())
                
    line = next(file)
            
    #print("opening cost route:")
    #print(opening_cost_route)
            
    line = next(file)
    zero_or_not = int(line.strip())
            
    #print("zero o uno: " + zero_or_not)
    return [n_customers, n_available_depots, v_capacity, opening_cost_route, zero_or_not]
    
def distance_cost(a1, a2, b1, b2):
    a1 = float(a1)
    a2 = float(a2)
    b1 = float(b1)
    b2 = float(b2)
    x1 = a1-b1
    x2 = a2-b2
    x1 = pow(x1, 2)
    x2 = pow(x2, 2)
    return math.sqrt(x1 + x2)

In [2]:
file_path = "Inst2"
start = time.time()
for filename in sorted(os.listdir(file_path), key=len):
    end = time.time()
    
    print("Tiempo desde el inicio del programa: " + str(end - start))
    print("Siguiente archivo")
    print(filename + "\n")
    f = os.path.join(file_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #Contenedores de info inicializados en nada
        n_customers = 0
        n_available_depots = 0
        coordinates_depots = []
        coordinates_customers = []
        v_capacity = 0
        capacities_depots = []
        customers_demands = []
        opening_costs_depots = []
        opening_cost_route = 0
        zero_or_not = 1
        
        #Acá se toma la info de los archivos .dat
        with open(f, 'r') as file:
            lista_var = []
            lista_var = fill_variables(filename, file_path, file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, 
                                       v_capacity, capacities_depots, customers_demands, opening_costs_depots, 
                                       opening_cost_route, zero_or_not)
            n_customers = lista_var[0]
            n_available_depots = lista_var[1]
            v_capacity = lista_var[2]
            opening_cost_route = lista_var[3]
            zero_or_not = lista_var[4]
        
        #Empezar operaciones acá
        #distancias_d2d = []  VIAJE ENTRE 2 DEPOTS NO PERMITIDO
        distancias_d2c = []
        distancias_c2c = []
        distancias_c2d = []
        distancias = []
        #Acá se consiguen los c_ij (costos de caminos)
        for i in coordinates_depots:
            '''for j in coordinates_depots:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2d.append(distance_cost(i[0], i[1], j[0], j[1]))'''
            for j in coordinates_customers:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2c.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_depots:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2d.append(distance_cost(i[0], i[1], j[0], j[1]))
                

        for i in range(len(coordinates_customers)):
            for j in range(len(coordinates_customers)):
                b = coordinates_customers[i]
                c = coordinates_customers[j]
                if j==i:
                    continue
                distancias.append(distance_cost(b[0], b[1], c[0], c[1]))
                distancias_c2c.append(distance_cost(b[0], b[1], c[0], c[1]))
                
        #MODELO CON CPLEX
        
        mdl = Model(name="MDRP")
        mdl.time_limit = 3600
        x = mdl.binary_var_list(len(distancias), name="x")
        
        #print(len(distancias))
        #C_ij * X_ij
        ab = [distancias[i]*x[i] for i in range(len(distancias))]
        mdl.minimize(mdl.sum(ab))
        
        #MODELO CON GUROBI
        '''
        modelo = Model("MDRP")
        x = modelo.addVars(len(distancias), vtype=GRB.BINARY, name="x")
        ab = [distancias[i]*x[i] for i in range(len(distancias))]
        print(modelo.setObjective(gp.sum(ab), GRB.MINIMIZE))
        '''
        
        #X_dj = yd con yd = 1
        for i in range(n_available_depots):
            offset = i*n_customers
            xdj = []
            for j in range(offset, offset+n_customers):
                xdj.append(x[j])
            mdl.add_constraint(sum(xdj) == 1)
            
        #X_jd = yd con yd = 1
        for i in range(n_available_depots):
            xjd = []
            for j in range(n_customers):
                offset = j*n_available_depots + n_available_depots*n_customers
                #print(str(offset) + " vs " + str(len(distancias)) + " vs " + str(len(x)))
                xjd.append(x[offset])
            mdl.add_constraint(sum(xjd) == 1)

            
        #X_ij = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[n_available_depots*i + j + len(distancias_d2c)])
            for j in range(n_customers-1):
                aux.append(x[(n_customers-1)*i + j + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)
        
        #X_ji = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[j*n_available_depots + i])
            for j in range(n_customers):
                if(j == i):
                    continue
                aux.append(x[j*(n_customers-1) + i + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)

        
        # DL para F-MDRP
        
        u = mdl.continuous_var_list(n_customers, name="u")
        for i in range(n_customers):
            offset_ij = (n_customers-1)*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = (n_customers-1)*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(u[i] - u[j] + (n_customers-n_available_depots+1)*x[j + offset_ij] + (n_customers-n_available_depots-1)*x[i + offset_ji]  <= n_customers - n_available_depots)
        
        '''
        # NODE-DL para MDRP
    
        k = mdl.continuous_var_list(n_customers, name="k")
        
        #Constraint 20
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[j*n_customers + i] <= n_available_depots)
                
        #Constraint 21
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[i*n_available_depots + j + len(distancias_d2c)] <= n_available_depots)
        
        #Constraint 22
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1)-1)*x[j*n_customers + i] <= k[i])
                
        #Constraint 23
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1) - 1)*x[i*n_available_depots + j + len(distancias_d2c)] <= k[i])
        
        #Constraint 24
        for i in range(n_customers):
            offset_ij = (n_customers-1)*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = (n_customers-1)*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(k[i] - k[j] <= (n_available_depots - 1)*(1 - x[j + offset_ij] - x[i + offset_ji]))
        '''
    
        '''
        #ARC
        g = mdl.continuous_var_list(len(x), name="g")
        
        #Constraint 27
        for i in range(n_available_depots):
            for j in range(n_customers):
                mdl.add_constraint(g[n_customers*i + j] == i*x[i*n_customers + j])
        #Constraint 28
        for i in range(n_customers):
            for j in range(n_available_depots):
                offset = len(distancias_d2c)
                mdl.add_constraint(g[n_available_depots*i + j + offset] == i*x[i*n_available_depots + j + offset])
        #Constraint 29
        for i in range(n_available_depots):
            for j in range(n_customers):
                ji_offset = len(distancias_d2c)
                g_ij = i*n_customers + j
                g_ji = ji_offset + j*n_available_depots + i
                mdl.add_constraint(g[g_ij] - g[g_ji] == 0)
                
        for i in range(n_customers):
            offset_ij = (n_customers-1)*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = (n_customers-1)*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(g[j + offset_ij] - g[i + offset_ji] == 0)
        #Constraint 30
        for i in range(len(x) - len(distancias_d2c) - len(distancias_c2d)):
            offset = len(distancias_d2c) + len(distancias_c2d)
            mdl.add_constraint(x[offset + i] <= g[offset + i])
            mdl.add_constraint(g[offset + i] <= i*x[offset + i])
            mdl.add_constraint(x[offset + i] <= i*x[offset + i])
            
        #SCF
        f = mdl.continuous_var_list(len(x) - len(distancias_c2d), name="f")
        f_aux = []
        
        #Constraint 11
        for i in range(len(distancias_d2c)):
            f_aux.append(f[i])
        mdl.add_constraint(sum(f_aux) == n_customers)
        
        #Constraint 12
        for i in range(n_customers):
            f_ji = []
            f_ij = []
            for j in range(n_available_depots):
                f_ji.append(f[j*n_customers + i])
            for j in range(n_customers):
                if(i==j):
                    continue
                offset = len(distancias_d2c)
                f_ji.append(f[j*(n_customers-1) + i + offset])
            for j in range(n_customers):
                if(i==j):
                    continue
                offset = len(distancias_d2c)
                f_ij.append(f[i*(n_customers-1) + j + offset])
            mdl.add_constraint(sum(f_ji) - sum(f_ij))
        
        #Constraint 15
        for i in range(n_available_depots):
            for j in range(n_customers):
                mdl.add_constraint(x[i*n_customers + j] <= f[i*n_customers + j])
                mdl.add_constraint(f[i*n_customers + j] <= (n_customers - n_available_depots + 1) * x[i*n_customers + j])
                
        #Constraint 16
        for i in range(n_customers):
            for j in range(n_customers):
                if i == j:
                    continue
                offset = len(distancias_d2c)
                mdl.add_constraint(x[i*(n_customers-1) + j + offset] <= f[i*(n_customers-1) + j + offset])
                mdl.add_constraint(f[i*(n_customers-1) + j + offset] <= (n_customers - n_available_depots)*x[i*(n_customers-1) + j + offset])
        '''
        
        #info de solución
        mdl.print_information()
        
        sol_start = time.time()
        solution = mdl.solve()
        sol_end = time.time()
    
        print("Tiempo en segundos: " + str(sol_end - sol_start))
        
        # Verificar si se encontró una solución
        if solution:
            # Extraer el valor óptimo de la función objetivo
            print(f"Valor óptimo de la función objetivo: {solution.objective_value}")
        else:
            print("No se encontró solución óptima.")
            
        print("Caminos usados: " + str(len(solution.get_value_list(x))))

        #solution.display()
        

Tiempo desde el inicio del programa: 0.0020003318786621094
Siguiente archivo
coordP111112.dat

Número de customers: 100
Número de depots disponibles: 10
Model: MDRP
 - number of variables: 12000
   - binary=11900, integer=0, continuous=100
 - number of constraints: 10120
   - linear=10120
 - parameters:
     parameters.timelimit = 3600.00000000000000
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 0.6019973754882812
Valor óptimo de la función objetivo: 744.6498421750834
Caminos usados: 11900
Tiempo desde el inicio del programa: 2.6711037158966064
Siguiente archivo
coordP111122.dat

Número de customers: 100
Número de depots disponibles: 20
Model: MDRP
 - number of variables: 14000
   - binary=13900, integer=0, continuous=100
 - number of constraints: 10140
   - linear=10140
 - parameters:
     parameters.timelimit = 3600.00000000000000
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 0.24100112915039062
Valor óptimo de la función objetivo: 689.0431

Model: MDRP
 - number of variables: 44000
   - binary=43800, integer=0, continuous=200
 - number of constraints: 40220
   - linear=40220
 - parameters:
     parameters.timelimit = 3600.00000000000000
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 43.360424518585205
Valor óptimo de la función objetivo: 682.3294090384579
Caminos usados: 43800
Tiempo desde el inicio del programa: 149.22887015342712
Siguiente archivo
coordP122122.dat

Número de customers: 200
Número de depots disponibles: 20
Model: MDRP
 - number of variables: 48000
   - binary=47800, integer=0, continuous=200
 - number of constraints: 40240
   - linear=40240
 - parameters:
     parameters.timelimit = 3600.00000000000000
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 13.926529884338379
Valor óptimo de la función objetivo: 722.2327375261332
Caminos usados: 47800
Tiempo desde el inicio del programa: 167.33476209640503
Siguiente archivo
coordP122212.dat

Número de customers: 200
Númer

Model: MDRP
 - number of variables: 25500
   - binary=25350, integer=0, continuous=150
 - number of constraints: 22670
   - linear=22670
 - parameters:
     parameters.timelimit = 3600.00000000000000
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 14.273548603057861
Valor óptimo de la función objetivo: 611.6984368364643
Caminos usados: 25350
Tiempo desde el inicio del programa: 354.54884123802185
Siguiente archivo
coordP133122.dat

Número de customers: 150
Número de depots disponibles: 20
Model: MDRP
 - number of variables: 28500
   - binary=28350, integer=0, continuous=150
 - number of constraints: 22690
   - linear=22690
 - parameters:
     parameters.timelimit = 3600.00000000000000
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 4.100933790206909
Valor óptimo de la función objetivo: 603.3756633052446
Caminos usados: 28350
Tiempo desde el inicio del programa: 361.05978655815125
Siguiente archivo
coordP133212.dat

Número de customers: 150
Número